In [ ]:
pip install happytransformer

  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=55847febb4f141aae5d0e5ffa9c1cd1208c32d160641cba37cfbe2a4a8fab6ae
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import csv
from happytransformer import TTSettings

In [ ]:
from happytransformer import HappyTextToText
from datasets import load_dataset
happ_tt=HappyTextToText("T5","t5-base")

train_dataset=load_dataset("jfleg",split="validation[:]")
test_dataset=load_dataset("jfleg",split="test[:]")
for case in train_dataset["corrections"][:2]:
    print(case)
    print(case[0])
    print("*"*100)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32100. This might induce some performance reduction as *Tensor Cores* will not be available. For more 

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
****************************************************************************************************
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
****************************************************************************************************


In [ ]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings.
                if input_text and correction:
                    writter.writerow([input_text, correction])



generate_csv("train.csv", train_dataset)
generate_csv("test.csv", test_dataset)

In [ ]:
b_result=happ_tt.eval("test.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
print(f"before loss:",b_result.loss)

before loss: 1.2803919315338135


In [ ]:
from happytransformer import TTTrainArgs
args=TTTrainArgs(batch_size=16)
happ_tt.train("train.csv",args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
1,1.564800,1.148065
17,0.876300,0.740885
34,0.832900,0.641610
51,0.725600,0.593207
68,0.678100,0.566035
85,0.674100,0.551391
102,0.636100,0.534813
119,0.653200,0.525637
136,0.608400,0.521119
153,0.592600,0.518975


In [ ]:
before_loss=happ_tt.eval("test.csv")
print(f"after_loss:",before_loss.loss)

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

after_loss: 0.502526581287384


In [ ]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
example_1 = "grammar: This sentences, has bads   grammar and spelling!"
result_1 = happ_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

